In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.master("local[*]").appName("SparkSQL").getOrCreate()

In [3]:

# ---- Implementation for no Hadoop pyspark total OS compatibility ----

# Read the CSV file from the URL into a Spark DataFrame
spark_df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

# ---- Original Implementation -----

# 1. Read in the AWS S3 bucket into a DataFrame.
#from pyspark import SparkFiles
#url = "s3a://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the CSV file to Spark's distributed cache using SparkFiles
#spark.sparkContext.addFile(url)

# Path to the CSV file within the distributed cache
#distributed_csv_path = SparkFiles.get(url)

# Read the CSV file from the distributed cache into a Spark DataFrame
#spark_df = spark.read.csv(distributed_csv_path, header=True, inferSchema=True)




# Show the contents of the DataFrame
spark_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.

# Create a temporary view named 'home_sales'
spark_df.createOrReplaceTempView("home_sales")

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("""
    SELECT date_built AS Date_Built, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales 
    WHERE bedrooms = 4 
    GROUP BY date_built 
    ORDER BY Date_Built, Average_Price
""")
result.show()
# Stop the Spark session

+----------+-------------+
|Date_Built|Average_Price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?


result = spark.sql("""SELECT
    date_built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales 
WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built""")

result.show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("""SELECT
    date_built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales 
WHERE bedrooms = 3 AND bathrooms = 3 AND floors= 2 AND sqft_living>=2000 GROUP BY date_built ORDER BY date_built""")

result.show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("""SELECT view AS views_rating, ROUND(AVG(price),2) as average_price FROM home_sales GROUP BY view HAVING ROUND(AVG(price),2)>=350000 ORDER BY view""")
result.show()
print("--- %s seconds ---" % (time.time() - start_time))

+------------+-------------+
|views_rating|average_price|
+------------+-------------+
|          51|    788128.21|
|          52|    733780.26|
|          53|     755214.8|
|          54|    798684.82|
|          55|    771153.32|
|          56|     718176.4|
|          57|     734340.5|
|          58|    759764.65|
|          59|     791453.0|
|          60|    754939.65|
|          61|    746877.59|
|          62|    759150.14|
|          63|    711614.55|
|          64|    767036.67|
|          65|    736679.93|
|          66|     712475.0|
|          67|    737970.96|
|          68|    716785.44|
|          69|    750537.94|
|          70|    695865.58|
+------------+-------------+
only showing top 20 rows

--- 1.7411401271820068 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.

spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [10]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached('home_sales')
print(is_cached)

True


In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result =spark.sql("SELECT view AS views_rating, ROUND(AVG(price),2) as average_price FROM home_sales GROUP BY view HAVING ROUND(AVG(price),2)>=350000 ORDER BY view")
result.show()
print("--- %s seconds ---" % (time.time() - start_time))

+------------+-------------+
|views_rating|average_price|
+------------+-------------+
|          51|    788128.21|
|          52|    733780.26|
|          53|     755214.8|
|          54|    798684.82|
|          55|    771153.32|
|          56|     718176.4|
|          57|     734340.5|
|          58|    759764.65|
|          59|     791453.0|
|          60|    754939.65|
|          61|    746877.59|
|          62|    759150.14|
|          63|    711614.55|
|          64|    767036.67|
|          65|    736679.93|
|          66|     712475.0|
|          67|    737970.96|
|          68|    716785.44|
|          69|    750537.94|
|          70|    695865.58|
+------------+-------------+
only showing top 20 rows

--- 1.2703278064727783 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


# # ---- Implementation for no Hadoop pyspark total OS compatibility ----

import pyarrow.parquet as pq
import pyarrow as pa

# convert spark dataframe to pandas dataframe then finally to arrow_table
arrow_table = pa.Table.from_pandas(spark_df.toPandas())

# create a arrow .parquet file of the arrow_table
pq.write_table(arrow_table, "home_sales.parquet")

# read parquet arrow table
arrow_table_parquet = pq.read_table("home_sales.parquet")

# convert parquet arrow table to pandas dataframe
parquet_arrow_pandas_df = arrow_table_parquet.to_pandas()

# set the index as date_built for partition in the pandas dataframe
parquet_arrow_pandas_df= parquet_arrow_pandas_df.set_index("date_built")

# save the pandas dataframe to parquet type with partition column and engine of type arrow
parquet_arrow_pandas_df.to_parquet("partitioned_home_sales.parquet", partition_cols=["date_built"], engine="pyarrow")

# ---------Original partitioning implementation--------------

# Save the  DataFrame as Parquet
#spark_df.write.parquet("home_sales.parquet")

# Load the formatted Parquet data
#spark_df_parquet = spark.read.parquet("home_sales.parquet")

# Partition the data by the date_built column and save it as a new Parquet file with partitioning
#spark_df_parquet.write.partitionBy("date_built").parquet("partitioned_home_sales.parquet")


In [13]:
# 11. Read the formatted parquet data.


# ---- Implementation for no Hadoop pyspark total OS compatibility ----

import os

# obtain path of each arrow partitions
partitions = [partition for partition in os.listdir("partitioned_home_sales.parquet") if os.path.isdir(os.path.join("partitioned_home_sales.parquet", partition))]

# create a object to unify all partitions into single one table 
merged_partitions = None

# Loop through each partition and read its contents
for partition in partitions:

    # get the full path alltogether for each partition
    partition_path = os.path.join("partitioned_home_sales.parquet", partition)

    # create an arrow table for the partition obtained
    arrow_table_partitioned = pq.read_table(partition_path)

    # convert that partitioned arrow table to pandas dataframe then finally to a spark dataframe
    spark_df_partitioned = spark.createDataFrame(arrow_table_partitioned.to_pandas())

    # check  merged_partitions intialization
    if merged_partitions is None:
            # assign the first partitioned spark dataframe
            merged_partitions = spark_df_partitioned
            continue
    # union each spark dataframe into a single partition
    merged_partitions = merged_partitions.union(spark_df_partitioned)

    # Optional print each partition
    print(f"Contents of partition {partition}:")
    spark_df_partitioned.show()

#----------- original implementation-----------

#spark_df_parquet_partitioned = spark.read.parquet("partitioned_home_sales.parquet")
#spark_df_parquet_partitioned.show()



Contents of partition date_built=2010:
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|8d54a71b-c520-44e...|2019-07-21|323956|       2|        3|       1506|   11816|     1|         0|  25|
|f5e01433-f7b0-44e...|2021-03-10|335326|       4|        3|       1588|   14107|     1|         0|   8|
|15a5830f-1529-4c5...|2019-07-06|843191|       3|        6|       5086|   14637|     2|         0|  91|
|63640c17-973d-4f5...|2019-11-14|148400|       2|        3|       2338|   11275|     2|         0|   5|
|209c1286-0834-463...|2021-07-04|395178|       3|        2|       1504|    9617|     2|         0|  37|
|bb2ff269-f08f-4e8...|2022-03-06|429030|       3|        3|       2049|   11882|     2|         0|  23|
|2d6d6575-aaf2-4ae...|202

In [14]:
# 12. Create a temporary table for the parquet data.


# ---- Implementation for no Hadoop pyspark total OS compatibility ----

merged_partitions.createOrReplaceTempView("partitioned_home_sales")

# ---------Original Implementation----------

#spark_df_parquet_partitioned.createOrReplaceTempView("partitioned_home_sales")



In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 


# as we are using the pandas partitioned type is not optimized to actual improve the time
# for the cached version, original implementation must be used to achieve this cases
# unfortunately time will be greather than original hadoop parquet partitioned data way to structure

start_time = time.time()

result = spark.sql("SELECT view AS views_rating, ROUND(AVG(price),2) as average_price FROM partitioned_home_sales GROUP BY view HAVING ROUND(AVG(price),2)>=350000 ORDER BY view")
result.show()
print("--- %s seconds ---" % (time.time() - start_time))

+------------+-------------+
|views_rating|average_price|
+------------+-------------+
|          51|    788128.21|
|          52|    733780.26|
|          53|     755214.8|
|          54|    798684.82|
|          55|    771153.32|
|          56|     718176.4|
|          57|     734340.5|
|          58|    759764.65|
|          59|     791453.0|
|          60|    754939.65|
|          61|    746877.59|
|          62|    759150.14|
|          63|    711614.55|
|          64|    767036.67|
|          65|    736679.93|
|          66|     712475.0|
|          67|    737970.96|
|          68|    716785.44|
|          69|    750537.94|
|          70|    695865.58|
+------------+-------------+
only showing top 20 rows

--- 166.80833864212036 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE Table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')
print(is_cached)
spark.stop()

False
